In [1]:
import numpy as np
import pandas as pd
import torch

from pathlib import Path

from models.StackedAutoEncoder import SimpleAutoEncoder, AutoEncoder, StackedAutoEncoder
from models.LSTM import LSTM

In [2]:
data = pd.read_excel(Path("RawData.xlsx"), "S&P500 Index Data")
features = list(data.columns[2:])
data[features] = data[features] / data[features].shift(1)
data.dropna(inplace=True)
train = data.loc[(data['Ntime'] >= 20120101) & (data['Ntime'] < 20140101)]
val = data.loc[(data['Ntime'] >= 20140101) & (data['Ntime'] < 20140401)]
test = data.loc[(data['Ntime'] >= 20140401) & (data['Ntime'] < 20140701)]
data = data[features]
data['target'] = data['Close Price'].shift(-1)
data.head()

,Close Price,Open Price,High Price,Low Price,Volume,MACD,CCI,ATR,BOLL,EMA20,MA10,MTM6,MA5,MTM12,ROC,SMI,WVAD,US Dollar Index,Federal Fund Rate,target
1,0.981796,1.007151,1.005337,1.000658,0.902468,1.079324,0.909298,1.244823,0.996465,0.995270,0.994167,1.594742,0.990626,1.312833,1.312727,1.531305,1.379623,0.995162,0.924171,1.001094
2,1.001094,0.982221,0.983988,0.992469,0.615530,1.042193,0.994015,0.635029,0.996334,0.995799,0.993853,1.119386,0.996830,0.892618,0.898704,0.689579,1.067612,1.010279,0.984615,0.991615
3,0.991615,0.999952,1.001943,0.990951,1.621332,1.049721,0.966405,1.708783,0.995803,0.995417,0.994922,0.441341,0.995906,0.971260,1.034488,1.193649,0.963357,0.999725,1.036458,1.017080
4,1.017080,0.991242,1.000173,1.001741,1.145988,0.969146,0.743315,0.941689,0.996342,0.997390,0.996555,0.241657,0.999007,0.808538,0.497853,0.619646,0.730327,1.003438,0.989950,0.977224
5,0.977224,1.017207,1.002504,1.001392,0.858619,1.042824,1.119407,1.046601,0.995914,0.995517,0.994568,6.384127,0.993651,1.059453,1.657384,1.892856,1.305448,0.994929,1.010152,1.006990


In [3]:
np.save("train.npy", train)
np.save("val.npy", val)
np.save("test.npy", test)

In [4]:
def split_data(stock, lookback):
    data_raw = stock.to_numpy()
    data = []
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    data = np.array(data)
    x = data[:, :-1]
    y = data[:, -1, :]
    x = torch.tensor(x, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)
    return x, y

x_train, y_train = split_data(train, lookback=20)
x_train.shape

torch.Size([482, 19, 21])

In [5]:
input_size = 1
hidden_size = 32
num_layers = 2
output_size = 1
dropout = 0.2
num_epochs = 10_000

model = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, output_size=output_size, dropout=dropout)
model.fit(x_train, y_train)

/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([482, 21])) that is different to the input size (torch.Size([482, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch  0 Loss:  19313918476288.0
Epoch  1 Loss:  19313916379136.0
Epoch  2 Loss:  19313910087680.0
Epoch  3 Loss:  19313905893376.0
Epoch  4 Loss:  19313905893376.0
Epoch  5 Loss:  19313903796224.0
Epoch  6 Loss:  19313903796224.0
Epoch  7 Loss:  19313899601920.0
Epoch  8 Loss:  19313895407616.0
Epoch  9 Loss:  19313895407616.0
Epoch  10 Loss:  19313891213312.0
Epoch  11 Loss:  19313889116160.0
Epoch  12 Loss:  19313889116160.0
Epoch  13 Loss:  19313880727552.0
Epoch  14 Loss:  19313880727552.0
Epoch  15 Loss:  19313880727552.0
Epoch  16 Loss:  19313876533248.0
Epoch  17 Loss:  19313872338944.0
Epoch  18 Loss:  19313872338944.0
Epoch  19 Loss:  19313866047488.0
Epoch  20 Loss:  19313861853184.0
Epoch  21 Loss:  19313861853184.0
Epoch  22 Loss:  19313861853184.0
Epoch  23 Loss:  19313853464576.0
Epoch  24 Loss:  19313853464576.0
Epoch  25 Loss:  19313851367424.0
Epoch  26 Loss:  19313849270272.0
Epoch  27 Loss:  19313849270272.0
Epoch  28 Loss:  19313845075968.0
Epoch  29 Loss:  1931383

KeyboardInterrupt: 